1. Importação das bibliotecas necessárias

In [1]:
import pandas as pd
import requests
import sqlite3 as sql
from plyer import notification 
from datetime import datetime

2. Criação de funções que serão utilizadas

2.1. Alerta de erro

In [2]:
'''
Esta é uma função de notificação de erro
'''

def exibir_alerta(nivel, base, etapa):

    now = str(datetime.now())
    msg = f'Falha no carregamento da base {base} na etapa {etapa}.\n{now}'

    if nivel == 1:
        title = 'ERRO! Alerta Baixo'
    elif nivel == 2:
        title = 'ERRO! Alerta Médio'
    elif nivel  == 3:
        title = 'ERRO! Alerta Alto'
    else:
        print(f'Nivel {nivel} não disponível!')


    notification.notify(
            title=title,
            message=msg,
            app_name='ETL_pipeline',
            timeout=10
        )
    

2.2. Obter dados da API

In [3]:
'''
Esta função será utilizada para extrair a API
'''

def extrair_api(url):

    resp = requests.get(url)

    if resp.status_code == 200:
        return resp.json()
    else:
        exibir_alerta(3, url, 'Extração')
        print(f'Falha ao buscar dados da API. Código do status: {resp.status_code}')
        return None
    

2.3. Criar um dataframe a partir do API extraído

In [4]:
def criar_dataframe(resp):

   resp_data_df = pd.DataFrame(resp)
   return resp_data_df


2.4. Transformar o dataframe em arquivo csv

In [5]:
def df_para_csv(dataframes, nomes_csv):

    for dataframe, nome_csv in zip(dataframes, nomes_csv):

        dataframe.to_csv(f'{nome_csv}.csv', index=False)
        print(f'Arquivo {nome_csv}.csv criado.')


2.5. Abrir conexão com a base de dados

In [6]:
def upload_to_db(df, db_name='../datas_core.db', table_name='nome_tabela'):
    
    conn = sql.connect(db_name)
    
    df.to_sql(table_name, conn, if_exists='replace', index=False)
   
    conn.close()
   
    print(f"Database {db_name} alimentada! Atualizada com a tabela {table_name}")
    

Função que abre conexão com a database e retorna como dataframe a tabela solicitada

In [7]:
def load_from_db(query, db_name = '../datas_core.db'):
    conn = sql.connect(db_name)
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

3. Extração e manipulação

3.1. Bancos

In [8]:
''' 
Visualização da tabela banks
'''

url_banks = 'https://brasilapi.com.br/api/banks/v1'
resp_data = extrair_api(url_banks)
df_bank = criar_dataframe(resp_data)
df_bank


,ispb,name,code,fullName
0,00000000,BCO DO BRASIL S.A.,1.0,Banco do Brasil S.A.
1,00000208,BRB - BCO DE BRASILIA S.A.,70.0,BRB - BANCO DE BRASILIA S.A.
2,00038121,Selic,NaN,Banco Central do Brasil - Selic
3,00038166,Bacen,NaN,Banco Central do Brasil
4,00122327,SANTINVEST S.A. - CFI,539.0,"SANTINVEST S.A. - CREDITO, FINANCIAMENTO E INV..."
...,...,...,...,...
354,92875780,WARREN CVMC LTDA,371.0,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...
355,92894922,BANCO ORIGINAL,212.0,Banco Original S.A.
356,94968518,EFX CC LTDA.,289.0,EFX CORRETORA DE CÂMBIO LTDA.
357,,NaN,NaN,NaN


In [9]:
'''
Visualização das informações do BD
'''
df_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359 entries, 0 to 358
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ispb      359 non-null    object 
 1   name      357 non-null    object 
 2   code      348 non-null    float64
 3   fullName  357 non-null    object 
dtypes: float64(1), object(3)
memory usage: 11.3+ KB


In [10]:
'''
Reorganizando e renomeando as colunas, removendo valores nulos
'''

banks_organizado = ['fullName', 'name', 'ispb', 'code']

df_bank = df_bank.reindex(columns=banks_organizado).dropna(subset=['fullName', 'name'])

df_bank.columns = ['Nome Completo', 'Nome', 'ISPB', 'Código']

df_bank = df_bank.sort_values(['Nome Completo'])

df_bank = df_bank.reset_index(drop=True)
df_bank


,Nome Completo,Nome,ISPB,Código
0,ACCREDITO - SOCIEDADE DE CRÉDITO DIRETO S.A.,ACCREDITO SCD S.A.,37715993,406.0
1,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...,13140088,332.0
2,ADVANCED CORRETORA DE CÂMBIO LTDA,ADVANCED CC LTDA,92856905,117.0
3,AGK CORRETORA DE CAMBIO S.A.,AGK CC S.A.,00250699,272.0
4,"AL5 S.A. CRÉDITO, FINANCIAMENTO E INVESTIMENTO",AL5 S.A. CFI,27214112,349.0
...,...,...,...,...
352,"ZEMA CRÉDITO, FINANCIAMENTO E INVESTIMENTO S/A",ZEMA CFI S/A,05351887,359.0
353,ZIPDIN SOLUÇÕES DIGITAIS SOCIEDADE DE CRÉDITO ...,ZIPDIN SCD S.A.,37414009,418.0
354,ÍNDIGO INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS ...,ÍNDIGO INVESTIMENTOS DTVM LTDA.,00329598,407.0
355,ÓTIMO SOCIEDADE DE CRÉDITO DIRETO S.A.,ÓTIMO SCD S.A.,34335592,355.0


3.2. PIX

In [11]:
''' 
Visualização da tabela pix
'''

url_pix = 'https://brasilapi.com.br/api/pix/v1/participants'
resp_data = extrair_api(url_pix)
df_pix = criar_dataframe(resp_data)
df_pix


,ispb,nome,nome_reduzido,modalidade_participacao,tipo_participacao,inicio_operacao
0,00000000,BANCO DO BRASIL S.A.,BCO DO BRASIL S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
1,00000208,BRB - BANCO DE BRASILIA S.A.,BRB - BCO DE BRASILIA S.A.,PDCT,DRCT,2020-11-03T09:30:00.000Z
2,00068987,COOPERATIVA DE CRÉDITO ARACREDI LTDA. - SICOOB...,CC ARACREDI LTDA.,PDCT,IDRT,2020-11-03T09:30:00.000Z
3,00075847,COOPERATIVA DE CRÉDITO UNICRED CENTRO-SUL LTDA...,CC UNICRED CENTRO-SUL LTDA - UNICRED CENTRO-SUL,PDCT,IDRT,2020-11-03T09:30:00.000Z
4,00106180,CENTRAL DAS COOPERATIVAS DE CRÉDITO DOS ESTADO...,"CCC DOS EST DE MT, MS E MUN DE CACOAL/RO",PDCT,IDRT,2020-11-03T09:30:00.000Z
...,...,...,...,...,...,...
821,95213211,"COOPERATIVA DE CRéDITO, POUPANçA E INVESTIMENT...",COOP SICREDI CAMINHO DAS ÁGUAS RS,PDCT,IDRT,2020-11-03T09:30:00.000Z
822,95424891,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI VALE DO RIO PARDO,PDCT,IDRT,2020-11-03T09:30:00.000Z
823,95594941,"COOPERATIVA DE CRÉDITO, POUPANÇA E INVESTIMENT...",COOP SICREDI REG CENTRO RS/MG,PDCT,IDRT,2020-11-03T09:30:00.000Z
824,97259253,COOPERATIVA E ECONOMIA E CRÉDITO MÚTUO DOS MÉD...,CECM MÉDICOS PLANALTO MÉDIO RS - CREDISIS/CRED...,PDCT,IDRT,2021-10-04T12:53:10.567Z


In [12]:
'''
Visualização das informações do BD
'''
df_pix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 826 entries, 0 to 825
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   ispb                     826 non-null    object
 1   nome                     826 non-null    object
 2   nome_reduzido            826 non-null    object
 3   modalidade_participacao  826 non-null    object
 4   tipo_participacao        826 non-null    object
 5   inicio_operacao          826 non-null    object
dtypes: object(6)
memory usage: 38.8+ KB


In [13]:
'''
Reorganizando e renomeando as colunas
'''

pix_organizado = ['nome', 'nome_reduzido', 'ispb', 'tipo_participacao', 'inicio_operacao']

df_pix = df_pix.reindex(columns=pix_organizado)

df_pix.columns = ['Nome Completo', 'Nome', 'ISPB', 'Tipo de Participação', 'Início da Operação']

df_pix = df_pix.sort_values(['Nome Completo'])

df_pix = df_pix.reset_index(drop=True)
df_pix


,Nome Completo,Nome,ISPB,Tipo de Participação,Início da Operação
0,7TRUST FINANCE INSTITUICAO DE PAGAMENTO S/A,7TRUST FINANCE & TAX ASSET,33930368,IDRT,2024-04-25T14:18:26.811Z
1,99PAY INSTITUICAO DE PAGAMENTO S.A.,99PAY IP S.A.,24313102,IDRT,2020-11-03T09:30:00.000Z
2,A27 INSTITUICAO DE PAGAMENTOS S/A,A27 IP S/A,35534511,IDRT,2024-03-11T13:11:19.684Z
3,ABASTECE AI CLUBE AUTOMOBILISTA INSTITUIÇÃO DE...,ABASTECE AI IP LTDA.,09515813,IDRT,2020-11-03T22:25:29.687Z
4,ACCREDITO - SOCIEDADE DE CRÉDITO DIRETO S.A.,ACCREDITO SCD S.A.,37715993,DRCT,2023-03-15T13:00:00.000Z
...,...,...,...,...,...
821,Z1 INSTITUIÇÃO DE PAGAMENTO LTDA.,Z1 IP LTDA.,35810871,DRCT,2024-03-19T13:10:00.000Z
822,"ZEMA CRÉDITO, FINANCIAMENTO E INVESTIMENTO S/A",ZEMA CFI S/A,05351887,DRCT,2023-01-23T12:00:00.000Z
823,ZOOP TECNOLOGIA & INSTITUICAO DE PAGAMENTO S.A.,ZOOP MEIOS DE PAGAMENTO,19468242,IDRT,2020-11-03T09:30:00.000Z
824,ZRO INSTITUIÇÃO DE PAGAMENTO S/A,ZRO IP S/A,26264220,IDRT,2023-11-17T15:11:31.362Z


3.3. Corretoras

In [14]:
''' 
Visualização da tabela corretoras
'''

url_corretoras = 'https://brasilapi.com.br/api/cvm/corretoras/v1'
resp_data = extrair_api(url_corretoras)
df_corretoras = criar_dataframe(resp_data)
df_corretoras

,cnpj,type,nome_social,nome_comercial,status,email,telefone,cep,pais,uf,municipio,bairro,complemento,logradouro,data_patrimonio_liquido,valor_patrimonio_liquido,codigo_cvm,data_inicio_situacao,data_registro
0,76621457000185,CORRETORAS,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,controle@4um.com.br,33519966,80420210,BRASIL,PR,CURITIBA,CENTRO,4º ANDAR,R. VISCONDE DO RIO BRANCO 1488,2005-12-31,4228660.18,2275,2006-10-05,1968-01-15
1,33817677000176,CORRETORAS,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL DTVM,CANCELADA,complianceregulatorio@abcbrasil.com.br,31702172,1453000,,SP,SÃO PAULO,ITAIM BIBI,2º ANDAR,"AV. CIDADE JARDIM, 803",2002-12-31,0.00,3514,2002-10-14,2002-10-14
2,10664027000132,CORRETORAS,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,,,50010240,BRASIL,PE,RECIFE,,,R DO IMP.D.PEDRO II 239/CJ.102,1989-12-31,5995252.29,329,1990-06-12,1986-07-08
3,44527444000155,CORRETORAS,ABN AMRO CLEARING CORRETORA DE T-TULOS E VALOR...,,EM FUNCIONAMENTO NORMAL,canal.comunicacao@br.abnamro.com,30737417,4551010,,SP,SÃO PAULO,VILA OLIMPIA,"11¦ ANDAR, CJ. 111","RUA FIDÛNCIO RAMOS, N¦ 302, TORRE B",2023-12-31,11122779.75,3794,2022-06-20,2022-06-20
4,33819590000138,CORRETORAS,AÇÃO S.A. CORRETORA DE VALORES E CÂMBIO,ACAO SA CVC,CANCELADA,,5095755,20040040,BRASIL,RJ,RIO DE JANEIRO,CENTRO,,"PRAÇA OLAVO BILAC, 28/ SL. 608",2000-12-31,291862.38,1112,2001-04-11,1968-04-19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,01170892000131,CORRETORAS,VOTORANTIM CTVM LTDA,VOTORANTIM,CANCELADA,demandasregulatorias@bancovotorantim.com.br,51711894,4794000,BRASIL,SP,SÃO PAULO,VL.GERTRUDES,TORRE A-14º ANDAR,"AV DAS NAÇÕES UNIDAS, 14.171",2017-12-31,266791832.77,3166,2018-08-27,1996-03-15
370,61769790000169,CORRETORAS,WALPIRES S/A CCTVM,WALPIRES S/A CCTVM,CANCELADA,andre@walpires.com.br,28087065,1452002,BRASIL,SP,SÃO PAULO,JARDIM PAULISTANO,1309 - 9ºANDAR,AVENIDA BRIGADEIRO FARIA LIMA,2017-12-31,7464373.90,1902,2019-09-24,1967-09-20
371,92875780000131,CORRETORAS,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,WARREN CORRETORA,EM FUNCIONAMENTO NORMAL,juridico@warren.com.br,32372273,90035191,,RS,PORTO ALEGRE,BOM FIM,2º ANDAR,"AVENIDA OSVALDO ARANHA, 720",2023-12-31,20672910.31,2534,1967-12-06,1967-12-06
372,24543688000114,CORRETORAS,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,PERFIL,CANCELADA,,2249022,52061540,BRASIL,PE,RECIFE,CASA FORTE,,"AV.17 DE AGOSTO,2579",1996-12-31,608260.02,2658,1997-02-21,1989-11-14


In [15]:
'''
Visualização das informações do BD
'''
df_corretoras.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 374 entries, 0 to 373
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   cnpj                      374 non-null    object
 1   type                      374 non-null    object
 2   nome_social               374 non-null    object
 3   nome_comercial            374 non-null    object
 4   status                    374 non-null    object
 5   email                     374 non-null    object
 6   telefone                  374 non-null    object
 7   cep                       374 non-null    object
 8   pais                      374 non-null    object
 9   uf                        374 non-null    object
 10  municipio                 374 non-null    object
 11  bairro                    374 non-null    object
 12  complemento               374 non-null    object
 13  logradouro                374 non-null    object
 14  data_patrimonio_liquido   

In [16]:
'''
Reorganizando e renomeando as colunas
'''

corretoras_organizado = ['nome_social', 'nome_comercial', 'status', 'uf', 'valor_patrimonio_liquido']

df_corretoras = df_corretoras.reindex(columns=corretoras_organizado)

df_corretoras.columns = ['Nome Completo', 'Nome', 'Status', 'Estado', 'Valor do Patrimônio Líquido']

df_corretoras = df_corretoras.sort_values(['Nome Completo'])

df_corretoras = df_corretoras.reset_index(drop=True)
df_corretoras


,Nome Completo,Nome,Status,Estado,Valor do Patrimônio Líquido
0,4UM DTVM S.A.,4UM INVESTIMENTOS,CANCELADA,PR,4228660.18
1,ABC BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES ...,ABC BRASIL DTVM,CANCELADA,SP,0.00
2,ABERTURA CCVM LTDA,ABERTURA CCVM LTDA,CANCELADA,PE,5995252.29
3,ABN AMRO CLEARING CORRETORA DE T-TULOS E VALOR...,,EM FUNCIONAMENTO NORMAL,SP,11122779.75
4,ACTION CCVM S.A.,ACTION CORRETORA,CANCELADA,SP,666907.71
...,...,...,...,...,...
369,VOTORANTIM CTVM LTDA,VOTORANTIM,CANCELADA,SP,266791832.77
370,WALPIRES S/A CCTVM,WALPIRES S/A CCTVM,CANCELADA,SP,7464373.90
371,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,WARREN CORRETORA,EM FUNCIONAMENTO NORMAL,RS,20672910.31
372,WARREN CORRETORA DE VALORES MOBILIÁRIOS E CÂMB...,PERFIL,CANCELADA,PE,608260.02


Criação de arquivos de extensão para csv a partir das dataframes

In [19]:
dataframes = [df_bank, df_pix, df_corretoras]
nomes_csv = ['df_banks', 'df_pix', 'df_corretoras']
df_para_csv(dataframes, nomes_csv)

Arquivo df_banks.csv criado.
Arquivo df_pix.csv criado.
Arquivo df_corretoras.csv criado.


ALIMENTAÇÃO DE DATABASE COM AS TABELAS CRIADAS

In [20]:
for dataframe, nome_tabela in zip(dataframes, nomes_csv):
    upload_to_db(dataframe, db_name= '../datas_core.db', table_name= nome_tabela)

Database ../datas_core.db alimentada! Atualizada com a tabela df_banks
Database ../datas_core.db alimentada! Atualizada com a tabela df_pix
Database ../datas_core.db alimentada! Atualizada com a tabela df_corretoras


In [22]:
query_banks = 'SELECT * FROM df_banks'
df_tabela_banks = load_from_db(query_banks)
df_tabela_banks

,Nome Completo,Nome,ISPB,Código
0,ACCREDITO - SOCIEDADE DE CRÉDITO DIRETO S.A.,ACCREDITO SCD S.A.,37715993,406.0
1,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...,ACESSO SOLUÇÕES DE PAGAMENTO S.A. - INSTITUIÇÃ...,13140088,332.0
2,ADVANCED CORRETORA DE CÂMBIO LTDA,ADVANCED CC LTDA,92856905,117.0
3,AGK CORRETORA DE CAMBIO S.A.,AGK CC S.A.,00250699,272.0
4,"AL5 S.A. CRÉDITO, FINANCIAMENTO E INVESTIMENTO",AL5 S.A. CFI,27214112,349.0
...,...,...,...,...
352,"ZEMA CRÉDITO, FINANCIAMENTO E INVESTIMENTO S/A",ZEMA CFI S/A,05351887,359.0
353,ZIPDIN SOLUÇÕES DIGITAIS SOCIEDADE DE CRÉDITO ...,ZIPDIN SCD S.A.,37414009,418.0
354,ÍNDIGO INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS ...,ÍNDIGO INVESTIMENTOS DTVM LTDA.,00329598,407.0
355,ÓTIMO SOCIEDADE DE CRÉDITO DIRETO S.A.,ÓTIMO SCD S.A.,34335592,355.0
